## 1. Import

In [50]:
import pandas as pd
import numpy as np

#StopWords Removal
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import re

import nltk

import textblob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Import Data Set

In [51]:
McDonalds_Df_Raw = pd.read_csv("McDonald_s_Reviews.csv", encoding="latin-1")
McDonalds_Df_Raw

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...,...,...,...,...,...,...,...,...
33391,33392,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1 star
33392,33393,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5 stars
33393,33394,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4 stars
33394,33395,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",5 years ago,"It's good, but lately it has become very expen...",5 stars


## 2. Organize Data

Remove unnecessary information

In [52]:
McDonalds_Df_Raw = McDonalds_Df_Raw.set_index("reviewer_id")
McDonalds_Df_Raw = McDonalds_Df_Raw.drop(columns = ["store_name", "category"])
McDonalds_Df_Raw

,store_address,latitude,longitude,rating_count,review_time,review,rating
reviewer_id,,,,,,,
1,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
2,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
3,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
4,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
5,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...,...,...,...,...,...
33392,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1 star
33393,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5 stars
33394,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4 stars


## 3. Keywords Frequency and Ratings

Select columns needed

In [53]:
df_keyword_ratings = McDonalds_Df_Raw.get(["review", "rating"])
df_keyword_ratings

,review,rating
reviewer_id,,
1,Why does it look like someone spit on my food?...,1 star
2,It'd McDonalds. It is what it is as far as the...,4 stars
3,Made a mobile order got to the speaker and che...,1 star
4,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
5,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...
33392,They treated me very badly.,1 star
33393,The service is very good,5 stars
33394,To remove hunger is enough,4 stars


Process dataframe

In [54]:
df_keyword_ratings = df_keyword_ratings.assign(rating = McDonalds_Df_Raw.get("rating").apply(lambda x: int(x[0])))
df_keyword_ratings = df_keyword_ratings.sort_values(by = ["rating"])
df_keyword_ratings

,review,rating
reviewer_id,,
1,Why does it look like someone spit on my food?...,1
13230,The do not offer free refills on your drinks i...,1
24392,"Terrible service, waited forever for our food,...",1
7067,Probably one of the worst experiences I have h...,1
7068,I paid 1 hotcakes and 1 hash brown by drive th...,1
...,...,...
24843,"Everything was cooked perfectly, and the Ser...",5
12364,Love it,5
24845,Im pretty sure that this is still the largest ...,5


4. Removing Stopwords

In [55]:

import nltk

stop_words = stopwords.words('english')
def remove_stop_words(review):
    '''
    Takes in a string comment or review returns a string that has the stopwords removed. For better comprehension of keywords.
    '''
    phrase = review.split()
    stripped_phrase = []
    for word in phrase:
        if word not in stop_words:
            stripped_phrase.append(word)

    return " ".join(stripped_phrase)


In [56]:
def remove_undecoded(review):
    '''
    removes the undecoded texts such as "�" in the original csv file. 
    '''
    return re.sub('[^0-9a-zA-Z\s]+', "",review)
    


In [57]:
#df_nostop_keywords = df_keyword_ratings.assign(no_stop_words_review = df_keyword_ratings.get("review").apply(remove_stop_words))
df_nostop_keywords = df_keyword_ratings.assign(no_stop_words_review = df_keyword_ratings.get("review").apply(remove_undecoded))
df_nostop_keywords = df_nostop_keywords.drop(columns = "review")

# Exporting the processed file into CSV
df_nostop_keywords.to_csv("review_without_stop_words", encoding = "latin-1")

In [58]:
# df to array put array
# import the training test split. 80 20, 60 40 70 30 <- use package
#
review = np.array(df_nostop_keywords.get("no_stop_words_review"))

In [59]:


def sentimentation(sentences):
    analyzer = SentimentIntensityAnalyzer()
    #compound_score = sentiment['compound']
    # Break down the text into sentences
    sentences = nltk.sent_tokenize(sentences)

    # Analyze sentiment for each sentence
    for sentence in sentences:
        sentiment = analyzer.polarity_scores(sentence)

    return sentiment['compound']

SyntaxError: 'return' outside function (2696330004.py, line 11)

TypeError: 'function' object is not subscriptable

In [ ]:
df_rating_1 = df_nostop_keywords[df_nostop_keywords.get("rating") == 1]
df_rating_1

,rating,no_stop_words_review
reviewer_id,,
1,1,Why does it look like someone spit on my food\...
13230,1,The do not offer free refills on your drinks i...
24392,1,Terrible service waited forever for our food W...
7067,1,Probably one of the worst experiences I have h...
7068,1,I paid 1 hotcakes and 1 hash brown by drive th...
...,...,...
16153,1,Terrible
3644,1,Not 24hrs They said it was 24 hours and they s...
3556,1,garbage


In [ ]:
np.random.seed(1)
df_sampled_1 = df_rating_1.sample(n=10)
df_sampled_1

,rating,no_stop_words_review
reviewer_id,,
9064,1,App wouldnt workdinning room supposed to be op...
10385,1,Infested with homeless people and crazy druggi...
32279,1,I cant believe how many times they ask me if I...
6354,1,This is the worst Mcdonalds I have ever been i...
18278,1,Ew omg the coke is so disgusting it does not t...
24531,1,If I could give this less than 1 star I absolu...
32112,1,If 0 was possible
6258,1,Dirty several people trying to sleep at tables...
23729,1,Loud speaker at drive thru Due to technical di...


In [ ]:
df_sampled_1.assign(sentiment = df_sampled_1.get("no_stop_words_review").apply(sentimentation))

,rating,no_stop_words_review,sentiment
reviewer_id,,,
9064,1,App wouldnt workdinning room supposed to be op...,"(-0.05, 0.3)"
10385,1,Infested with homeless people and crazy druggi...,"(-0.4333333333333333, 0.6555555555555556)"
32279,1,I cant believe how many times they ask me if I...,"(0.15, 0.6)"
6354,1,This is the worst Mcdonalds I have ever been i...,"(0.023487394957983212, 0.5558963585434173)"
18278,1,Ew omg the coke is so disgusting it does not t...,"(-1.0, 1.0)"
24531,1,If I could give this less than 1 star I absolu...,"(-0.10095238095238093, 0.5009523809523809)"
32112,1,If 0 was possible,"(0.0, 1.0)"
6258,1,Dirty several people trying to sleep at tables...,"(-0.3, 0.55)"
23729,1,Loud speaker at drive thru Due to technical di...,"(-0.05654761904761905, 0.468452380952381)"


In [ ]:
df_rating_2 = df_nostop_keywords[df_nostop_keywords.get("rating") == 2]
df_rating_2

,rating,no_stop_words_review
reviewer_id,,
11045,2,Artisan Grilled Chicken was like rubber Could ...
16663,2,Worst food ever The hamburgers taste like a paper
19269,2,The workers there are not professionali went i...
17115,2,Poor
24989,2,The service is not slow but the customer servi...
...,...,...
26382,2,No Refills It took about 10 minutes for me an...
26336,2,The locations looks great and has 8 of machine...
32893,2,This place is always getting the order wrong


In [ ]:
np.random.seed(1)
df_sampled_2 = df_rating_2.sample(n=10)
df_sampled_2

,rating,no_stop_words_review
reviewer_id,,
15801,2,Shake machine ALWAYS DOWN after Lunch Rush NEV...
26867,2,Bad customer service rude they lie about givin...
22474,2,Very long time waiting to make a order Very la...
7897,2,Horrible I hate to have to say that because I ...
31867,2,Ice cream machine doww to early
25758,2,They must have been busy before we got there c...
32815,2,Food never fresh employee gets mad when you as...
31344,2,I like that it is animal theme and has a playg...
25603,2,Food was cold as their heart And the classic t...


In [ ]:
df_sampled_2.assign(sentiment = df_sampled_2.get("no_stop_words_review").apply(sentimentation))

,rating,no_stop_words_review,sentiment
reviewer_id,,,
15801,2,Shake machine ALWAYS DOWN after Lunch Rush NEV...,"(-0.0851851851851852, 0.12962962962962962)"
26867,2,Bad customer service rude they lie about givin...,"(-0.3999999999999999, 0.4388888888888889)"
22474,2,Very long time waiting to make a order Very la...,"(0.10678571428571429, 0.5385714285714286)"
7897,2,Horrible I hate to have to say that because I ...,"(-0.2329931972789116, 0.5637755102040817)"
31867,2,Ice cream machine doww to early,"(0.1, 0.3)"
25758,2,They must have been busy before we got there c...,"(-0.45, 0.65)"
32815,2,Food never fresh employee gets mad when you as...,"(-0.3875, 0.75)"
31344,2,I like that it is animal theme and has a playg...,"(-0.2878571428571428, 0.5971428571428572)"
25603,2,Food was cold as their heart And the classic t...,"(-0.21666666666666667, 0.5833333333333334)"
